# Commands

In [ ]:
# jupython --venv st-venv ~/notebooks/DeepLearning/asamin-dual.ipynb

In [ ]:
# titanv 1:
# screen -S asamin-dual1
# source ~/.bash_profile ; source ~/.bash_aliases ; cd ~/misc-logs
# DOCKER_PORT=9961 nn -o nohup-dual-$HOSTNAME-1.out ~/docker/keras/run-jupython.sh ~/notebooks/asa/eval/asamin-dual.ipynb titanv
# observe ~/misc-logs/nohup-dual-$HOSTNAME-1.out

In [ ]:
# titanv 2:
# screen -S asamin-dual2
# source ~/.bash_profile ; source ~/.bash_aliases ; cd ~/misc-logs
# DOCKER_PORT=9962 nn -o nohup-dual-$HOSTNAME-2.out ~/docker/keras/run-jupython.sh ~/notebooks/asa/eval/asamin-dual.ipynb titanv
# observe ~/misc-logs/nohup-dual-$HOSTNAME-2.out

# Init

In [ ]:
from machinelearning.seeder import *
seed()

In [ ]:
isNotebook = '__file__' not in locals()

In [ ]:
TEST = False # isNotebook, False, True

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Imports

In [ ]:
from newssource.asattribution.utils import *
from newssource.asattribution.asamin import *
from newssource.asa.asapreproc import *
from newssource.asa.models import *
from newssource.metrics.ndcg import *

In [ ]:
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
if not isNotebook:
    matplotlib.use('Agg')
import random
import time
import pickle
import copy
from hashlib import md5

In [ ]:
from keras import backend as K
K.tensorflow_backend._get_available_gpus()

In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, LSTM, Flatten
from keras.layers.embeddings import Embedding
from keras.models import load_model
from keras.utils import multi_gpu_model

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn.dummy import DummyClassifier
from sklearn.model_selection import KFold, StratifiedKFold

In [ ]:
from gensim.test.utils import common_texts
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

In [ ]:
from random import random
from numpy import array
from numpy import cumsum
from keras.models import Sequential
from keras.layers import LSTM, GRU
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.utils import multi_gpu_model

In [ ]:
import statistics

In [ ]:
from machinelearning.baseline import *
from machinelearning.encoder import *
from machinelearning.kerasutils import *
from machinelearning.kerasmodels import *

In [ ]:
from machinelearning.baseline import *
from machinelearning.encoder import *
from machinelearning import kerasutils
from machinelearning.iterator import *
from machinelearning.metrics import *

In [ ]:
from keras.layers import concatenate, Input
from keras.models import Model
from keras.utils import plot_model

In [ ]:
import scipy

In [ ]:
from keras.layers import LSTM, GRU, Dense, CuDNNLSTM, CuDNNGRU, TimeDistributed, Flatten, concatenate
from keras.utils import multi_gpu_model, plot_model
from keras.layers import concatenate, Input
from keras.models import Model, load_model, Sequential
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.layers.embeddings import Embedding
from keras.callbacks import Callback, History, ModelCheckpoint

# Modules reloading

In [ ]:
from importlib import reload

In [ ]:
def reloadKerasutils():
    global kerasutils
    kerasutils = reload(kerasutils)

# Misc vars

In [ ]:
logger = Logger(tmpDir() + "/asamin-dual.log" if hjlat() else "asamin-dual.log")

In [ ]:
tt = TicToc(logger=logger)
tt.tic()

# Results

In [ ]:
results = dict()

In [ ]:
def addResult(obj, *args, doPrint=True):
    global config
    global results
    if isinstance(obj, float):
        obj = {"score": obj}
    if len(args) == 1:
        if isinstance(args[0], list):
            path = args[0]
        else:
            path = [args]
    else:
        path = args
    currentResult = results
    for key in path[:-1]:
        if key not in currentResult:
            currentResult[key] = dict()
        currentResult = currentResult[key]
    key = path[-1]
    if key not in currentResult:
        currentResult[key] = []
    currentResult = currentResult[key]
    localConf = copy.deepcopy(config)
    for k in ["doNotif", "doMultiGPU", "doFlattenSentences", "filterNonWordOrPunct", "punct", "outputDir"]:
        if k in localConf:
            del localConf[k]
    currentResult.append(mergeDicts(obj, {"config": localConf}))
    if doPrint:
        printResults()

In [ ]:
def printResults():
    global results
    global logger
    localRes = copy.deepcopy(results)
    def recDel(d):
        if isinstance(d, dict):
            if "config" in d:
                del d["config"]
            for key in d.keys():
                d[key] = recDel(d[key])
        elif isinstance(d, list):
            for i in range(len(d)):
                d[i] = recDel(d[i])
        return d
    localRes = recDel(localRes)
    log("results without configs:\n" + lts(localRes), logger)

In [ ]:
def saveState():
    toJsonFile(mongoStorable(config), config["outputDir"] + "/config.json")
    toJsonFile(results, config["outputDir"] + "/results.json")

# Config

In [ ]:
config = dict()

In [ ]:
config = mergeDicts(config, \
{
    # <IMPORTANT>
    "dataCol": "filtered_sentences", # filtered_sentences, sentences, usetSentences, 3gramsFiltered, 2gramsFiltered, 1gramsFiltered, textSentences
    "minTokensLength": 3,
    "minVocDF": 2,
    "minVocLF": 2,
    
    # <IMPORTANT>
    # "docLength": 1200,
    
    # <IMPORTANT>
    # "wordVectorsPattern": "test", # glove-6B, fasttext, glove-840B.300d
    # "embeddingsDimension": 100,
    # "doLower": True,
    
    # <IMPORTANT>
    # ASA2:
    "transfer": {"dirName": "6ebdd3e05d4388c658ca2d5c53b0bc36", "epoch": 35, "aaPop": 3, "srLayerId": -8}, # filtered_sentences, epochs 35 (???) 33 (???)
    # "transfer": {"dirName": "7fa22619af09e90724d2e3a8cf5db796", "epoch": 26, "aaPop": 3, "srLayerId": -8}, # textSentences

    # <OTHERS>
    # "scoring": "accuracy",
    # "inputEncoding": "index", # index, embedding # <IMPORTANT>
    # "cv": 2 if TEST else 10,
    # "patience": 0 if TEST else 30, # 30
    # "batchSize": 32 if TEST else 32, # 128 doesn't work here, don't know why...
    # "doNotif": not isNotebook,
    
    # uset0-l50-dpl50-d18-bc10, asa, bchack (others: blogcorpus, c50)
    # uset0-l50-dpl200-d18-bc10, uset1-l50-dpl200-d18-bc10, uset2-l50-dpl200-d18-bc10, uset3-l50-dpl200-d18-bc10, uset4-l50-dpl200-d18-bc10
    # uset0-l50-dpl160-blogger.com, uset1-l50-dpl160-blogger.com, uset2-l50-dpl160-blogger.com, uset3-l50-dpl160-blogger.com, uset4-l50-dpl160-blogger.com
    "datasetName": "uset4-l50-dpl160-blogger.com",
    
    
    "usePretrained": True,
    "batchSize": 128,
    "patience": 60,
    "epochs": 500,
    "loss": 'categorical_crossentropy', # sparse_categorical_crossentropy, categorical_crossentropy
    "metrics": ['accuracy', 'top_k_categorical_accuracy'],
    "saveMetrics":
    {
        "val_loss": "min",
        "val_acc": "max",
        "val_top_k_categorical_accuracy": "max",
    },
    "dropout": 0.2,
    "denseUnits": 100,
})

In [ ]:
config["outputDir"] = nosaveDir() + "/asa2-focus/" + objectToHash(config)

In [ ]:
if TEST:
    (dir, filename, ext, filenameExt) = decomposePath(config["outputDir"])
    config["outputDir"] = dir + "/tests/" + filenameExt
mkdir(config["outputDir"])
logger = Logger(config["outputDir"] + "/asa2-focus.log")

In [ ]:
log(lts(config), logger)

# We prepare data

In [ ]:
dataRootDir = dataDir() + "/Asa2/usets"
dataDirectory = dataRootDir + "/" + config["datasetName"]
asapFiles = sortedGlob(dataDirectory + "/*.bz2")
bp(asapFiles)

In [ ]:
attTransferDir = nosaveDir() + "/asa2-train/" + config["transfer"]["dirName"]
attTransferEpochPath = attTransferDir + "/models/epoch" + digitalizeIntegers(str(config["transfer"]["epoch"]), 4)
attTransferWeightsPath = attTransferEpochPath + "/weights.h5"
attTransferPrebuilt = deserialize(attTransferDir + "/asap-prebuilt.pickle")
attTransferConfig = fromJsonFile(attTransferDir + "/config.json")
attTransferDocLength = attTransferConfig["docLength"]

In [ ]:
asapKwargs = \
{
    'dataCol': config["dataCol"],
    'minTokensLength': config["minTokensLength"],
    'minVocDF': config["minVocDF"],
    'minVocLF': config["minVocLF"],
    'persist': [True],
    'docLength': attTransferConfig["docLength"],
    'labelEncoding': "onehot",
    'logger': logger,
}
asap = buildASAP(asapFiles, **asapKwargs)

In [ ]:
attTransferPrebuilt["labelEncoder"] = asap.labelEncoder
attTransferPrebuilt["samplesCounts"] = asap.samplesCounts

In [ ]:
attTransferAsapKwargs = \
{
    'dataCol': config["dataCol"],
    'minTokensLength': config["minTokensLength"],
    # 'batchSize': config["batchSize"],
    'minVocDF': config["minVocDF"],
    'minVocLF': config["minVocLF"],
    'persist': [True],
    'prebuilt': attTransferPrebuilt,
    'logger': logger,
    'verbose': True,
    
    'docLength': attTransferDocLength,
}
attTransferAsapKwargs["encoding"] = attTransferConfig["inputEncoding"]
attTransferAsap = buildASAP(asapFiles, **attTransferAsapKwargs)

In [ ]:
attTransferModelKwargs = fromJsonFile(attTransferEpochPath + "/kwargs.json")
assert attTransferModelKwargs["vocSize"] == len(attTransferPrebuilt["vocIndex"])
assert attTransferModelKwargs["docLength"] == attTransferAsap.getDocLength()
assert attTransferModelKwargs["vocSize"] == len(attTransferAsap.getVocIndex())

In [ ]:
docs = [row["filtered_sentences"] for row in NDJson(asapFiles[0])]    

In [ ]:
bp(docs)

In [ ]:
splitIndex = int(0.75 * len(docs))
trainDocs = docs[:splitIndex]
testDocs = docs[splitIndex:]
log(str(len(trainDocs)) + " docs in the training set and " + str(len(testDocs)) + " docs in the test set", logger)

# Init of DeepStyle

In [ ]:
from deepstyle.model import *

In [ ]:
# modelName = "6ebdd3e05d4388c658ca2d5c53b0bc36"
modelName = "extrafocus-model"

In [ ]:
model = DeepStyle(nosaveDir() + "/asa2-train/" + modelName, logger=logger)

# We get attentions

In [ ]:
attentions = model.attentions(testDocs, progressVerbose=True)
# model.save()
bp(attentions, logger)

In [ ]:
def truncateAttentions(attentions, docs):
    newAttentions = []
    for i in range(len(docs)):
        # Getting the attention of the current doc:
        doc = docs[i]
        attention = attentions[i]
        docLength = len(attention)
        # Unpadding the attention:
        if len(doc) < docLength:
            pad = docLength - len(doc)
            attention = attention[pad:]
        # We check the shape:
        if len(doc) < docLength:
            assert len(attention) == len(doc)
        else:
            assert len(attention) == docLength
        # Making it a proba distrib:
        attention = toProbDist(attention)
        # And finally add it to attentions:
        newAttentions.append(attention)
    return newAttentions

In [ ]:
attDocs = [x[0] for x in attTransferAsap.getRawPart(truncate=True, pad=False)]
attTestDocs = attDocs[splitIndex:]

In [ ]:
assert len(attTestDocs) == len(attentions)
attentions = truncateAttentions(attentions, attTestDocs)

# We get TFIDF values

In [ ]:
tfidfDocs = [flattenLists(doc)[:1200] for doc in docs]
bp(tfidfDocs, 4, logger)

In [ ]:
tfidf = TFIDF(tfidfDocs, logger=logger, doLower=True, sublinearTF=True)

In [ ]:
tfidfValues = tfidf.getTFIDFVectors()

In [ ]:
tfidfValues = tfidfValues[splitIndex:]

In [ ]:
for i in range(len(tfidfValues)):
    tfidfValues[i] = toProbDist(tfidfValues[i])

In [ ]:
assert len(tfidfValues) == len(attentions)

In [ ]:
for i in range(len(tfidfValues)):
    assert len(tfidfValues[i]) == len(attentions[i])

In [ ]:
bp(tfidfValues, logger)

# We get the attention focus

In [ ]:
def getAttentionFocus(attentions, tfidfValues):
    attentionFocus = []
    for i in range(len(attentions)):
        attention = attentions[i]
        tfidf = tfidfValues[i]
        attentionFocus.append(np.dot(attention, tfidf))
    return np.mean(attentionFocus)

In [ ]:
attentionFocus = getAttentionFocus(attentions, tfidfValues)

In [ ]:
attentionFocus = attentionFocus * 100

In [ ]:
log("attention focus: " + str(attentionFocus), logger)